## residential charging

In [ ]:
energy_use_all_residential = pd.DataFrame()
for pr in [10,25,50,75,100]:
    energy_use = pd.DataFrame([],columns=['zone','hour','energy_use'])
    energy_use['zone'] = np.repeat(zones,24)
    energy_use['zone'] = energy_use['zone'].astype('int64')
    energy_use['hour'] = list(range(24)) * len(zones)
    energy_use['energy_use'] = 0

    lambda_E = 0.35
    E_max = 30
    power_E = 6.7
    trajectory_df = []
    total_vehicles = 0
    total_trips = 0
    penetration_rate = pr

    for k in range(len(zones)):
        init_o = int(zones[k])
        
        trajectory_df = pd.read_feather(data_dir+'OD/Clarke County_v3/trip_chain_'+str(init_o)+'.feather')
        total_vehicles = total_vehicles+len(np.unique(trajectory_df.veh_id))
        total_trips = total_trips+len(trajectory_df.trip_id)
        
        # assume home charging 100%
        # get the EV label of each car (for now assume 100%)
        nvehicles = len(np.unique(trajectory_df.veh_id))

        id_assignment_dict = dict(zip(np.unique(trajectory_df.veh_id), 
                                      np.random.choice([0, 1], 
                                                       size=100, 
                                                       p=[1 - penetration_rate/100, penetration_rate/100])))
        trajectory_df['EV_label'] = trajectory_df['veh_id'].map(id_assignment_dict)

        trajectory_df['E_start'] = 0
        trajectory_df['E_end'] = 0
        for j in range(1, nvehicles+1):
            df = trajectory_df[trajectory_df.veh_id==j]
            if df['EV_label'].values[0] == 0:
                continue
            
            df.loc[df.trip_id==1,'E_start'] = E_max
            df.loc[df.trip_id==1,'E_end'] = E_max - lambda_E * df.loc[df.trip_id==1,'distance'].values[0]
            ntrips = len(np.unique(df.trip_id))
            if ntrips > 1:
                for i in range(2,ntrips+2):  
                    E_left = df.loc[df.trip_id==i-1,'E_end'].values[0]                  
                    
                    if  (E_left < 1 *E_max) & ((df.loc[df.trip_id==i-1,'purpose'].values[0]=='home')): # | (i==ntrips+1)                     
                        # charging e consumption at previous destination
                        time_max = (E_max-df.loc[df.trip_id==i-1,'E_end'].values[0])/power_E
                        stay_time = max(0,df.loc[df.trip_id==i-1,'stay_time'].values[0])
                        charging_time = min(stay_time,time_max)
                        
                        # charging start time
                        charging_start_time = int(df.loc[df.trip_id==i-1,'arrival_time'])
                        charging_end_time = charging_start_time+charging_time

                        # charging location:
                        end_zone = df.loc[df.trip_id==i-1,'end_zone'].values[0]
                        
                        # for each hour of charging:
                        if charging_time<1:
                            t = charging_start_time
                            if t>=24:
                                t = t-24
                            energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] = energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] + power_E * charging_time
                        else:
                            for t in np.arange(charging_start_time, np.ceil(charging_end_time), 1):
                                if t>=24:
                                    t = t-24
                                energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] = energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] + power_E * 1
                            min_left = charging_end_time-(np.ceil(charging_end_time)-1)
                            energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] = energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] + power_E * min_left
                        

                        if i<ntrips+1:                      
                            # start of this trip
                            E_start = df.loc[df.trip_id==i-1,'E_end'].values[0] + power_E * charging_time
                            df.loc[df.trip_id==i,'E_start'] = min(E_start,E_max)
                            df.loc[df.trip_id==i,'E_end'] = max(0,df.loc[df.trip_id==i,'E_start'].values[0] - lambda_E * df.loc[df.trip_id==i,'distance'].values[0])
                    else:
                        if i<ntrips+1: 
                            df.loc[df.trip_id==i,'E_start'] = df.loc[df.trip_id==i-1,'E_end'].values[0]
                            df.loc[df.trip_id==i,'E_end'] = max(0,df.loc[df.trip_id==i,'E_start'].values[0] - lambda_E * df.loc[df.trip_id==i,'distance'].values[0])                 
 
    # energy use need to be scaled by population
    energy_use = pd.merge(energy_use,bg_base,on=['zone'])
    # calibrate simulation demand
    energy_use['energy_use'] = energy_use['energy_use'] * energy_use['base_veh']/100
    energy_use['penetration rate'] = pr
    energy_use_all_residential = pd.concat([energy_use_all_residential,energy_use])

In [ ]:
summary = energy_use_all_residential.groupby(['hour','penetration rate']).agg({'energy_use':'sum'}).reset_index()
# summary.loc[summary.hour.isin([21,22,23,0,1,2,3]),'energy_use'].sum()/summary['energy_use'].sum()
summary.head()

min_val, max_val = 0.3,1.0
n = 5
custom_palette = sns.color_palette("YlOrBr", n_colors=n, as_cmap=True)
custom_palette = custom_palette(np.linspace(min_val, max_val, n))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("mycmap", custom_palette)

sns.lineplot(data=summary,x='hour',y='energy_use',hue='penetration rate',marker="o"
             , palette=cmap)# ,color='steelblue'
plt.legend(title='Penetration Rate (%)')
plt.ylabel('Energy use at Clarke County (kW)')
plt.xlabel('Time of day')
plt.ylim(0,40000)
plt.show()

In [ ]:
summary[summary.energy_use==np.max(summary.energy_use)]

In [ ]:
second_highest_energy = np.partition(summary['energy_use'], -3)[-3]
summary.loc[summary['energy_use'] == second_highest_energy]
# summary[summary.energy_use==np.max(summary.energy_use)]

## function - charge everywhere

In [ ]:
energy_use_all = pd.DataFrame()
for pr in [10,25,50,75,100]:
    energy_use = pd.DataFrame([],columns=['zone','hour','energy_use'])
    energy_use['zone'] = np.repeat(zones,24)
    energy_use['zone'] = energy_use['zone'].astype('int64')
    energy_use['hour'] = list(range(24)) * len(zones)
    energy_use['energy_use'] = 0

    lambda_E = 0.35 #  energy consumption of EV kWh/mi
    E_max = 30 # battery capacity kWh - need to consider brand
    power_E = 6.7 # charging power kWh/h - need to consider slow/fast
    trajectory_df = []
    total_vehicles = 0
    total_trips = 0
    penetration_rate = pr

    for k in range(len(zones)):
        init_o = int(zones[k])
        
        trajectory_df = pd.read_feather(data_dir+'OD/Clarke County_v3/trip_chain_'+str(init_o)+'.feather')
        total_vehicles = total_vehicles+len(np.unique(trajectory_df.veh_id))
        total_trips = total_trips+len(trajectory_df.trip_id)
        
        # assume home charging 100%
        # get the EV label of each car (for now assume 100%)
        nvehicles = len(np.unique(trajectory_df.veh_id))

        id_assignment_dict = dict(zip(np.unique(trajectory_df['veh_id']), 
                                      np.random.choice([0, 1], 
                                                       size=100, 
                                                       p=[1 - penetration_rate/100, penetration_rate/100])))
        trajectory_df['EV_label'] = trajectory_df['veh_id'].map(id_assignment_dict)

        trajectory_df['E_start'] = 0
        trajectory_df['E_end'] = 0
        for j in range(1, nvehicles+1):
            df = trajectory_df[trajectory_df.veh_id==j]
            if df['EV_label'].values[0] == 0:
                continue
            
            df.loc[df.trip_id==1,'E_start'] = E_max
            df.loc[df.trip_id==1,'E_end'] = E_max - lambda_E * df.loc[df.trip_id==1,'distance'].values[0]
            ntrips = len(np.unique(df.trip_id))
            if ntrips > 1:
                for i in range(2,ntrips+2):  
                    E_left = df.loc[df.trip_id==i-1,'E_end'].values[0] # some trips cannot be finished due to not enough battery
                    if  (E_left < 1 * E_max):
                        # charging consumption at previous destination
                        time_max = (E_max-df.loc[df.trip_id==i-1,'E_end'].values[0])/power_E # in hour
                        stay_time = max(0,df.loc[df.trip_id==i-1,'stay_time'].values[0])
                        charging_time = min(stay_time,time_max)
                        
                        # charging start time
                        charging_start_time = int(df.loc[df.trip_id==i-1,'arrival_time'])
                        charging_end_time = charging_start_time + charging_time
                        
                        # charging location:
                        end_zone = df.loc[df.trip_id==i-1,'end_zone'].values[0]
                        
                        # for each hour of charging:
                        if charging_time<1:
                            t = charging_start_time
                            energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] = energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] + power_E * charging_time
                        else:
                            for t in np.arange(charging_start_time, np.ceil(charging_end_time), 1):                               
                                if t>=24:
                                    t = t-24
                                energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] = energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] + power_E * 1
                            min_left = charging_end_time-(np.ceil(charging_end_time)-1)
                            energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] = energy_use.loc[(energy_use.hour==t)&(energy_use.zone==end_zone),'energy_use'] + power_E * (charging_end_time-t)

                        if i<ntrips+1:
                            # start of charge of this trip
                            E_start = df.loc[df.trip_id==i-1,'E_end'].values[0] + power_E * charging_time
                            df.loc[df.trip_id==i,'E_start'] = min(E_start,E_max)
                            df.loc[df.trip_id==i,'E_end'] = max(0,df.loc[df.trip_id==i,'E_start'].values[0] - lambda_E * df.loc[df.trip_id==i,'distance'].values[0])

                    else:
                        if i<ntrips+1:
                            df.loc[df.trip_id==i,'E_start'] = df.loc[df.trip_id==i-1,'E_end'].values[0]
                            df.loc[df.trip_id==i,'E_end'] = max(0,df.loc[df.trip_id==i,'E_start'].values[0] - lambda_E * df.loc[df.trip_id==i,'distance'].values[0])

    # energy use need to be scaled by population
    energy_use = pd.merge(energy_use,bg_base,on=['zone'])
    # calibrate simulation demand
    energy_use['energy_use'] = energy_use['energy_use'] * energy_use['base_veh']/100
    energy_use['penetration rate'] = pr
    energy_use_all = pd.concat([energy_use_all,energy_use])

In [ ]:
summary = energy_use_all.groupby(['hour','penetration rate']).agg({'energy_use':'sum'}).reset_index()
# summary.loc[summary.hour.isin([21,22,23,0,1,2,3]),'energy_use'].sum()/summary['energy_use'].sum()
summary.head()

min_val, max_val = 0.3,1.0
n = 5
custom_palette = sns.color_palette("YlOrBr", n_colors=n, as_cmap=True)
custom_palette = custom_palette(np.linspace(min_val, max_val, n))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("mycmap", custom_palette)

sns.lineplot(data=summary,x='hour',y='energy_use',hue='penetration rate',marker="o"
             , palette=cmap)# ,color='steelblue'
plt.legend(title='Penetration Rate (%)')
plt.ylabel('Energy use at Clarke County (kW)')
plt.xlabel('Time of day')
plt.ylim(0,40000)
plt.show()

In [ ]:
print(energy_use_all.energy_use.sum())
print(energy_use_all_residential.energy_use.sum())
energy_use_all_residential[energy_use_all_residential.energy_use==np.max(energy_use_all_residential.energy_use)]